<a href="https://colab.research.google.com/github/RedietNegash/TrainingMaterials/blob/gp/N_Queens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Understanding**
- We need to place N queens on an N x N chessboard such that no two queens can attack each other.
  This means that no two queens should share the same row, column, or diagonal.

- Our goal is to find a valid arrangement of queens that meets the above conditions.To achieve a solution efficiently using a genetic algorithm that can find the arrangement within a reasonable number of generations.



# **Approach**
We will solve this problem using a Genetic Algorithm.


**1. Chromosome Representation**
Each chromosome represents a potential solution to the problem, where each position in the chromosome indicates the row of a queen in a specific column.

**2. Fitness Function**
The fitness function evaluates how well a chromosome solves the N-Queens problem. It calculates the number of collisions (both horizontal and diagonal) between queens, with a higher fitness score indicating fewer collisions. The maximum fitness score is given, which represents the best possible arrangement.

**3. Selection**
A probabilistic selection process is used to choose chromosomes for reproduction based on their fitness scores. This ensures that fitter chromosomes have a higher chance of being selected for the next generation.

**4. Crossover and Mutation**
- **Crossover:** Two parent chromosomes are combined to produce a child chromosome. A random crossover point is selected, and the genes from both parents are joined to create the child.
- **Mutation:** With a small probability, a queen’s position in a chromosome is randomly changed. This introduces genetic diversity, which can help avoid local minima.

**5. Evolution Process**
The algorithm iterates through generations, repeatedly applying selection, crossover, and mutation to produce new populations until a solution is found or a predefined condition is met.


# **Constraints:**
- **Problem Size:** The algorithm's performance may degrade with larger values of N due to increased complexity in evaluating fitness and generating solutions. The maximum fitness value is also dependent on N.
  
- **Randomness:** The stochastic nature of genetic algorithms means that results may vary between runs. While the algorithm is designed to converge towards a solution, there's no guarantee that it will always find an optimal arrangement within a limited number of generations.

- **Mutation Probability:** The mutation rate is set to a low value (e.g., 3%), which may affect the algorithm's ability to explore the solution space effectively. A higher mutation rate could increase exploration but may also disrupt good solutions.

- **Resource Limitations:** Depending on the implementation environment, memory and processing power might limit the size of N that can be effectively handled.



**Code**

In [3]:
import random

def random_chromosome(size):
    """
       Generate a random chromosome representing positions of queens on an N x N chessboard, where each position is represented by a number from 1 to N,
       indicating the row of the queen in each column.
    """
    return [random.randint(1, nq) for _ in range(size)]

def fitness(chromosome):
    """
       Calculate the fitness score of a chromosome by counting the number of collisions between queens,
       including horizontal and diagonal collisions, to evaluate how close the chromosome is to a valid solution with maximum fitness representing no collisions.

    """
    horizontal_collisions = sum([chromosome.count(queen) - 1 for queen in chromosome]) / 2
    diagonal_collisions = 0
    n = len(chromosome)
    left_diagonal = [0] * (2 * n)
    right_diagonal = [0] * (2 * n)

    for i in range(n):
        left_diagonal[i + chromosome[i] - 1] += 1
        right_diagonal[len(chromosome) - i + chromosome[i] - 2] += 1

    for i in range(2 * n - 1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i] - 1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i] - 1
        diagonal_collisions += counter / (n - abs(i - n + 1))

    return int(maxFitness - (horizontal_collisions + diagonal_collisions))

def probability(chromosome, fitness):
    """
        Calculate the selection probability of a chromosome based on its fitness score, which reflects the chromosome's
        ability to avoid collisions, allowing for preferential selection of fitter chromosomes in the population.

    """
    return fitness(chromosome) / maxFitness

def random_pick(population, probabilities):
    """
        Select a chromosome from the population based on their calculated probabilities, ensuring that chromosomes with higher fitness
        have a greater chance of being chosen for reproduction, thereby guiding the evolutionary process.

    """
    total = sum(probabilities)
    r = random.uniform(0, total)
    upto = 0

    for chrom, prob in zip(population, probabilities):
        if upto + prob >= r:
            return chrom
        upto += prob

    assert False, "Shouldn't get here"

def reproduce(parent1, parent2):
    """
       Create a child chromosome by combining segments of two parent chromosomes at a randomly chosen crossover point,
        allowing the child to inherit traits from both parents to enhance its fitness potential.
    """
    n = len(parent1)
    crossover_point = random.randint(0, n - 1)
    return parent1[0:crossover_point] + parent2[crossover_point:n]

def mutate(chromosome):
    """
       Introduce variability in a chromosome by randomly changing the position of one queen, which can help explore
       new solutions and avoid local optima in the genetic algorithm's search space.

    """
    n = len(chromosome)
    mutation_index = random.randint(0, n - 1)
    new_position = random.randint(1, n)
    chromosome[mutation_index] = new_position
    return chromosome

def genetic_queen(population, fitness):
    """
       Run one generation of the genetic algorithm for the N-Queens problem, creating a new population by selecting parents,
       performing crossover and mutation, and aiming to find a solution with maximum fitness representing no collisions among queens.
    """
    mutation_probability = 0.03
    new_population = []
    probabilities = [probability(chrom, fitness) for chrom in population]

    for _ in range(len(population)):
        parent1 = random_pick(population, probabilities)
        parent2 = random_pick(population, probabilities)
        child = reproduce(parent1, parent2)

        if random.random() < mutation_probability:
            child = mutate(child)

        print_chromosome(child)
        new_population.append(child)

        if fitness(child) == maxFitness:
            break

    return new_population

def print_chromosome(chromosome):
    """
       Display the chromosome and its corresponding fitness score, providing insight into the current
       state of the genetic algorithm's search for a solution to the N-Queens problem.

    """
    print("Chromosome = {},  Fitness = {}".format(str(chromosome), fitness(chromosome)))

if __name__ == "__main__":
    nq = int(input("Enter Number of Queens: "))
    maxFitness = (nq * (nq - 1)) // 2
    population = [random_chromosome(nq) for _ in range(100)]

    generation = 1

    while maxFitness not in [fitness(chrom) for chrom in population]:
        print("=== Generation {} ===".format(generation))
        population = genetic_queen(population, fitness)
        print("")
        print("Maximum Fitness = {}".format(max(fitness(chrom) for chrom in population)))
        generation += 1

    print("Solved in Generation {}!".format(generation - 1))


    # Find and display the solution chromosome
    for chrom in population:
        if fitness(chrom) == maxFitness:
            print("\nOne of the solutions: ")
            print_chromosome(chrom)

    # Prepare the board for display
    board = [["x"] * nq for _ in range(nq)]

    for i in range(nq):
        board[nq - chrom[i]][i] = "Q"  # Place queens on the board

    def print_board(board):
        """Print the chessboard representation of the solution."""
        for row in board:
            print(" ".join(row))

    print()
    print_board(board)

Enter Number of Queens: 4
=== Generation 1 ===
Chromosome = [4, 3, 3, 1],  Fitness = 4
Chromosome = [4, 4, 4, 1],  Fitness = 2
Chromosome = [2, 3, 1, 4],  Fitness = 5
Chromosome = [3, 2, 4, 3],  Fitness = 4
Chromosome = [3, 4, 2, 2],  Fitness = 4
Chromosome = [3, 4, 1, 2],  Fitness = 4
Chromosome = [2, 4, 3, 4],  Fitness = 4
Chromosome = [4, 1, 4, 1],  Fitness = 3
Chromosome = [4, 4, 4, 2],  Fitness = 2
Chromosome = [4, 3, 2, 2],  Fitness = 4
Chromosome = [1, 1, 2, 4],  Fitness = 4
Chromosome = [4, 3, 3, 4],  Fitness = 3
Chromosome = [3, 2, 4, 3],  Fitness = 4
Chromosome = [4, 3, 1, 4],  Fitness = 4
Chromosome = [1, 1, 2, 2],  Fitness = 3
Chromosome = [3, 1, 1, 4],  Fitness = 4
Chromosome = [2, 4, 2, 1],  Fitness = 4
Chromosome = [1, 1, 4, 1],  Fitness = 3
Chromosome = [4, 3, 4, 1],  Fitness = 4
Chromosome = [2, 4, 3, 4],  Fitness = 4
Chromosome = [3, 2, 3, 3],  Fitness = 2
Chromosome = [1, 4, 3, 1],  Fitness = 4
Chromosome = [3, 4, 2, 2],  Fitness = 4
Chromosome = [4, 3, 3, 3],  Fitne

we set the number of queens to **4**. The algorithm began by randomly generating an initial population of chromosomes, each representing a possible arrangement of queens on a 4x4 chessboard.

- **Generation 1**:
  - The algorithm evaluated the fitness of various chromosomes, with the best fitness score being **5**.

- **Generation 2**:
  - The algorithm continued to evolve the population, producing better arrangements.
  - The highest fitness score achieved in this generation was **6**.

After **7 generations**, the algorithm successfully found a solution with a fitness score of **6**, indicating no collisions among the queens. The final arrangement of queens is represented by the chromosome `[3, 1, 4, 2]`.

### Board Representation
The final solution is visually represented on the chessboard as follows:

```
x x Q x
Q x x x
x x x Q
x Q x x
```

In this arrangement, "Q" represents the position of each queen, while "x" indicates empty squares. The algorithm's performance demonstrated a successful application of genetic algorithms to the N-Queens problem, efficiently navigating through generations to arrive at a valid solution.